In [37]:
import pandas as pd
import numpy as np
import copy
import sklearn 
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
import sklearn.metrics as metrics 

# from sklearn.exceptions import DeprecationWarning
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [38]:
## create test dataset
# A,B,Y = XOR probelm
# C = noise 
a_0 = np.full((2), 0)
a_1 = np.full((2), 1)
a = np.concatenate((a_0, a_1), axis = 0)
a_final = np.tile(a, 2500)

b_0 = np.full((1), 0)
b_1 = np.full((1), 1)
b = np.concatenate((b_0, b_1), axis = 0)
b_final = np.tile(b, 5000)

c_final = np.random.randint(2, size = 10000)
np.random.shuffle(c_final)
print(sum(c_final))
# c_0 = np.full((5000), 0)
# c_1 = np.full((5000), 1)
# c_final = np.concatenate((c_0, c_1), axis = 0)
# np.random.shuffle(c_final)

y_0 = np.array([0,1])
y_1 = np.array([1,0])
y = np.concatenate((y_0, y_1), axis = 0)
y_final = np.tile(y, 2500)

4948


In [39]:
df_data = pd.DataFrame(index=range(10000), columns=['A','B','C','Y'])

In [40]:
df_data['A'] = a_final
df_data['B'] = b_final
df_data['C'] = c_final
df_data['Y'] = y_final

In [41]:
df_data.head()

,A,B,C,Y
0,0,0,0,0
1,0,1,0,1
2,1,0,0,1
3,1,1,1,0
4,0,0,0,0


In [42]:
df_data = df_data.sample(frac=1).reset_index(drop=True)
df_data[0:10]

,A,B,C,Y
0,0,0,1,0
1,0,1,0,1
2,1,0,0,1
3,0,0,0,0
4,0,0,1,0
5,0,1,1,1
6,0,1,0,1
7,0,0,1,0
8,0,1,1,1
9,1,0,1,1


In [43]:
df_features = df_data[['A','B','C']]
df_target = df_data[['Y']]

## TEST EM

In [59]:
# split train test 
X_train, X_test, Y_train, Y_test = train_test_split(df_features, df_target, stratify=df_target, test_size=0.3)

In [60]:
X_train.shape

(7000, 3)

In [61]:
X_test.shape

(3000, 3)

In [62]:
data = X_train.as_matrix()
print(data.shape)

target = (Y_train.as_matrix()).ravel()
print(target.shape)

test_data = X_test.as_matrix()
print(test_data.shape)

test_target = (Y_test.as_matrix()).ravel()
print(test_target.shape)

(7000, 3)
(7000,)
(3000, 3)
(3000,)


In [63]:
data[0:5]

array([[1, 1, 0],
       [0, 1, 0],
       [1, 1, 0],
       [0, 0, 1],
       [0, 1, 1]])

In [64]:
target[0:5]

array([0, 1, 0, 0, 1])

In [65]:
test_model = LR(data, target)
test_model.score(test_data, test_target)

0.51800000000000002

In [66]:
# randomly initialize cohort 
# cohort = np.random.randint(2, size=7000)
# cohort[0:10]

# initalize cohort by A
cohort = copy.deepcopy(data[:,0])
cohort

array([1, 0, 1, ..., 1, 1, 0])

In [67]:
index_counter = 0
num_change = 0
while True:
    data_1 = data[np.where(cohort == 0)]
    print(data_1.shape)
    target_1 = target[np.where(cohort == 0)]
    print(target_1.shape)


    data_2 = data[np.where(cohort == 1)]
    target_2 = target[np.where(cohort == 1)]
    print(target_2.shape)


    cohort1_Model = LR(data_1, target_1)

    cohort2_Model = LR(data_2, target_2)
    
    print('ITERATION:', index_counter)
    cohort1_pred = cohort1_Model.predict(data_1) #reshape(-1,1)
    cohort1_current_accuracy = metrics.accuracy_score(target_1, cohort1_pred)
    print('Cohort 1 Accuracy:', cohort1_current_accuracy*100, '%')
    print("Majority Class:", (1 - (sum(target_1)/len(target_1)))*100,'%')
    print('cohort size:', (len(target_1)/len(target))*100,'%')
    print(" ")
    
    cohort2_pred = cohort2_Model.predict(data_2) #reshape(-1,1)
    cohort2_current_accuracy = metrics.accuracy_score(target_2, cohort2_pred)
    print('Cohort 2 Accuracy:', cohort2_current_accuracy*100, '%')
    print("Majority Class", (1 - (sum(target_2)/len(target_2)))*100,'%')
    print('cohort size:', (len(target_2)/len(target))*100,'%')
    print(" ")
    print(" ")
    print(" ")

    oldcohort = copy.deepcopy(cohort)
    
    # reassign based on whichever model gives higher probability
    for i in range(len(target)):
        curr_data = data[i]
        if (target[i] == 0):  
            if np.any((cohort1_Model.predict_proba(curr_data)[:,0]) > (cohort2_Model.predict_proba(curr_data)[:,0])):
                cohort[i] = 0
            else:
                cohort[i] = 1
        else:
            if np.any((cohort1_Model.predict_proba(curr_data)[:,1]) > (cohort2_Model.predict_proba(curr_data)[:,1])):
                cohort[i] = 0
            else:
                cohort[i] = 1
                    
    if sum(np.absolute(np.subtract(oldcohort, cohort))) == 0 or index_counter == 10:
        num_change = sum(np.absolute(np.subtract(oldcohort, cohort)))
        print('num_change', num_change)
        print("break")
        break
    else:
        num_change = sum(np.absolute(np.subtract(oldcohort, cohort)))
        print('num_change', num_change)
    index_counter += 1

(3532, 3)
(3532,)
(3468,)
ITERATION: 0
Cohort 1 Accuracy: 100.0 %
Majority Class: 49.8301245753 %
cohort size: 50.457142857142856 %
 
Cohort 2 Accuracy: 100.0 %
Majority Class 50.1730103806 %
cohort size: 49.542857142857144 %
 
 
 
num_change 0
break


In [83]:
a = 100
b = 11
c = 1111

In [85]:
if a > (b and c):
    print('yes')
else:
    print('no')

no


In [68]:
cohort=cohort.astype('int')
cohort_clf = LR(data, cohort)

In [69]:
test_cohort = cohort_clf.predict(X_test)

test_cohort[0:100]

array([1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0])

In [70]:
testdata_1 = test_data[np.where(test_cohort == 0)]
testtarget_1 = test_target[np.where(test_cohort == 0)]

testdata_2 = test_data[np.where(test_cohort == 1)]
testtarget_2 = test_target[np.where(test_cohort == 1)]

In [74]:
test_cohort1_pred = cohort1_Model.predict(testdata_1)
print('Accuracy', (metrics.accuracy_score(testtarget_1, test_cohort1_pred))*100,'%')
print("Majority Class", (1 - (sum(testtarget_1)/len(testtarget_1)))*100,'%')
print('cohort size:', (len(testtarget_1)/len(test_target))*100,'%')

test_cohort2_pred = cohort2_Model.predict(testdata_2)
print('Accuracy', (metrics.accuracy_score(testtarget_2, test_cohort2_pred))*100,'%')
print("Majority Class", (1 - (sum(testtarget_2)/len(testtarget_2)))*100,'%')
print('cohort size:', (len(testtarget_2)/len(test_target)*100),'%')

Accuracy 100.0 %
Majority Class 50.408719346 %
cohort size: 48.93333333333334 %
Accuracy 100.0 %
Majority Class 49.6083550914 %
cohort size: 51.06666666666667 %


In [75]:
testdata_1[0:50]

array([[0, 0, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 1],
       [0, 1, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 1],
       [0, 0, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 1],
       [0, 1, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 1],
       [0, 1, 1]])

In [76]:
testdata_2[0:50]

array([[1, 1, 0],
       [1, 0, 0],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 1],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 1, 0],
       [1, 1, 0],
       [1, 0, 0],
       [1, 1, 1],
       [1, 1, 0],
       [1, 1, 0],
       [1, 1, 1],
       [1, 0, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 0],
       [1, 1, 0],
       [1, 0, 0],
       [1, 1, 1],
       [1, 0, 1],
       [1, 0, 1],
       [1, 0, 1],
       [1, 1, 1],
       [1, 1, 0],
       [1, 1, 0],
       [1, 1, 0],
       [1, 0, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 0],
       [1, 0, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 1],
       [1, 0, 0],
       [1, 1, 0],
       [1, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 1],
       [1, 0, 1],
       [1, 1, 0]])

In [36]:
def LR(features, target):
    """
    INPUT
        features
        target
    
    OUTPUT
        optimized clf
    """

    # Create regularization penalty space
    penalty = ['l1', 'l2']

    # Create regularization hyperparameter space
    C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]


    # Create hyperparameter options
    hyperparameters = dict(C=C, penalty=penalty)
    
    clf = GridSearchCV(estimator=LogisticRegression(), 
                    param_grid=hyperparameters,
                    cv=5,
                    refit=True)

    clf.fit(features, target)
    optimised_logistic_regression = clf.best_estimator_
    return optimised_logistic_regression
    